In [ ]:
from datasets import load_dataset
from transformers import AutoModelForTokenClassification, AutoTokenizer, PreTrainedTokenizerFast


ss_dataset = load_dataset("lamm-mit/protein_secondary_structure_from_PDB", split="train", streaming=True)

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.12/dist-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]


In [2]:
model = AutoModelForTokenClassification.from_pretrained("example_8m_checkpoint", num_labels=8, trust_remote_code=True)

Some weights of NVEsmForTokenClassification were not initialized from the model checkpoint at example_8m_checkpoint and are newly initialized: ['classifier._extra_state', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
entry = next(iter(ss_dataset))

In [4]:
ss_tokenizer = PreTrainedTokenizerFast(tokenizer_file="ss_tokenizer.json")
aa_tokenizer = AutoTokenizer.from_pretrained("example_8m_checkpoint")

tokenizer_args = {
    "max_length": 1024,
    "truncation": True,
    # "stride": 100,  # figure this out later
    # "return_overflowing_tokens": True,
}

In [ ]:
def tokenize(example):
    result = {}
    result["input_ids"] = aa_tokenizer(example["Sequence"], **tokenizer_args)["input_ids"]
    tokenized_labels = ss_tokenizer(example["Secondary_structure"], **tokenizer_args)["input_ids"]
    result["labels"] = [[ii if ii != 8 else -100 for ii in item] for item in tokenized_labels]
    return result

In [8]:
tokenized_dataset = ss_dataset.map(
    tokenize, batched=True, remove_columns=[col for col in ss_dataset.features if col not in ["input_ids", "labels"]]
)

In [9]:
from transformers import DataCollatorForTokenClassification

In [10]:
collator = DataCollatorForTokenClassification(tokenizer=aa_tokenizer, padding="max_length", max_length=1024)

In [11]:
import torch

In [12]:
dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=16, collate_fn=collator)
next(iter(dataloader))

ValueError: Column lengths mismatch: columns ['labels'] have length [493556] while input_ids has length 1000.